In [111]:
import uproot
import numpy as np
import torch
from collections import defaultdict
from util import get_layer, theta_func, calculate_num_pixels_z_dependence,create_layer_map
import matplotlib.pyplot as plot

layer_map, super_layer_map = create_layer_map()
x_hit_pos = []
z_pos_list = []
layer_list = []
def process_root_file(file_path):
    print("began processing")
    with uproot.open(file_path) as file:
        tree = file["events/HcalBarrelHits"]
        
        z_pos = tree["HcalBarrelHits.position.z"].array(library="np")
        x_pos = tree["HcalBarrelHits.position.x"].array(library="np")
        energy = tree["HcalBarrelHits.EDep"].array(library="np")
        momentum_x = tree["HcalBarrelHits.momentum.x"].array(library="np")
        momentum_y = tree["HcalBarrelHits.momentum.y"].array(library="np")
        momentum_z = tree["HcalBarrelHits.momentum.z"].array(library="np")
        hit_time = tree["HcalBarrelHits.time"].array(library="np")
        print("finished loading branches")
        processed_data = defaultdict(dict)
        
        for event_idx in range(len(z_pos)):
            energy_per_layer = defaultdict(float)
            first_hit_per_layer = {}
            
            # First pass: collect first hit data and calculate energy per layer
            for hit_idx in range(len(z_pos[event_idx])):
                z = z_pos[event_idx][hit_idx]
                z_pos_list.append(z)
                x = x_pos[event_idx][hit_idx]
                x_hit_pos.append(x)
                e = energy[event_idx][hit_idx]
                momentum = (momentum_x[event_idx][hit_idx],
                            momentum_y[event_idx][hit_idx],
                            momentum_z[event_idx][hit_idx])
                momentum_mag = np.linalg.norm(momentum)
                theta = theta_func(momentum_x[event_idx][hit_idx],momentum_y[event_idx][hit_idx],momentum_z[event_idx][hit_idx])
                layer = get_layer(x)
                layer_list.append(layer)
                
                energy_per_layer[layer] += e
                
                if layer not in first_hit_per_layer:
                    first_hit_per_layer[layer] = {
                        "z_pos": z,
                        "x_pos": x,
                        "energy": e,
                        "momentum": momentum_mag,
                        "theta": theta,
                        "time": hit_time[event_idx][hit_idx]
                    }
            
            # Second pass: process first hit with total layer energy
            for layer, hit_data in first_hit_per_layer.items():
                layer_energy = energy_per_layer[layer]
                num_pixels = calculate_num_pixels_z_dependence(layer_energy,hit_data["z_pos"])
                hit_data["num_pixels"] = num_pixels
                hit_data["layer_energy"] = layer_energy  # Store total layer energy for reference
                processed_data[event_idx][layer] = hit_data
    print("finished processing")
    return processed_data

def prepare_nn_input(processed_data, normalizing_flow):
    nn_input = defaultdict(lambda: defaultdict(list))
    nn_output = defaultdict(lambda: defaultdict(list))
    
    for event_idx, event_data in processed_data.items():
        for layer, hit_data in event_data.items():
            context = torch.tensor([hit_data['z_pos'], hit_data['theta'], hit_data['momentum']], dtype=torch.float32)
            context = context.repeat(hit_data['num_pixels'], 1)  # Repeat context for each pixel
            
            # Generate samples for all pixels in the layer
            with torch.no_grad():
                samples = normalizing_flow.sample(num_samples=1, context=context).squeeze()
            
            # Add the sampled times to the original time
            adjusted_times = samples + hit_data['time']
            
            # Sort all pixels by adjusted time and take the lowest 10
            sorted_indices = torch.argsort(adjusted_times)[:10]
            
            nn_input[event_idx][layer] = adjusted_times[sorted_indices]
            nn_output[event_idx][layer] = torch.full((10,), hit_data['momentum'], dtype=torch.float32)
    
    return nn_input, nn_output

def prepare_prediction_input(nn_input, nn_output):
    prediction_input = []
    prediction_output = []
    
    for event_idx in nn_input.keys():
        event_input = []
        event_output = []
        
        for layer in sorted(nn_input[event_idx].keys()):
            layer_times = nn_input[event_idx][layer]
            layer_momenta = nn_output[event_idx][layer]
            
            # Pad or truncate to exactly 10 times per layer
            if len(layer_times) < 10:
                padding = torch.full((10 - len(layer_times),), float('inf'))
                layer_times = torch.cat([layer_times, padding])
            
            event_input.append(layer_times)
            event_output.append(layer_momenta.mean())  # Using mean momentum as the target
        
        prediction_input.append(torch.stack(event_input))
        prediction_output.append(torch.stack(event_output))
    
    return torch.stack(prediction_input), torch.stack(prediction_output)

# Example usage:
# def calculate_theta(z, x):
#     return np.arctan2(x, z)

# def calculate_num_pixels(z, x, layer_energy):
#     # Implement your logic to calculate the number of pixels for a hit
#     # using z, x positions and the total energy deposited in the layer
#     pass
pref = "/hpc/group/vossenlab/rck32/"
# processed_data = process_root_file(pref + "eic/work_eic/root_files/September_3/sector_scint/run_1_pip_0_8_10GeV_theta_90_500events.edm4hep.root")
# processed_data = process_root_file(pref + "eic/work_eic/root_files/September_3/sector_scint/run_1_mum_0_8_10GeV_theta_90_500events.edm4hep.root")
processed_data = process_root_file(pref + "eic/work_eic/root_files/September_3/sector_scint/run_1_n_0_8_10GeV_theta_90_5kevents.edm4hep.root")
# nn_input, nn_output = prepare_nn_input(processed_data, normalizing_flow_model)
# prediction_input, prediction_output = prepare_prediction_input(nn_input, nn_output)

began processing
finished loading branches
finished processing


In [114]:
sensor_file = uproot.open(pref + "eic/work_eic/root_files/September_3/sector_sensor/run_1_mum_0_8_10GeV_theta_90_20events.edm4hep.root:events")

In [128]:
len(sensor_file["HcalBarrelHits/HcalBarrelHits.position.x"].array(library = "np")[2]) / 28

69.5

In [129]:
len(sensor_file["MCParticles/MCParticles.PDG"].array(library = "np")[2]) / 28

71.10714285714286

In [106]:
calculate_num_pixels_z_dependence(3.73e-6,-11)

17.07690897410014

In [110]:
processed_data[0]

{0: {'z_pos': np.float64(-0.09267735751465048),
  'x_pos': np.float64(1830.8),
  'energy': np.float32(0.0018906922),
  'momentum': np.float32(1.097112),
  'theta': np.float32(90.176056),
  'time': np.float32(6.1350164),
  'num_pixels': np.float64(14790.31562969877),
  'layer_energy': np.float32(0.0031699687)},
 1: {'z_pos': np.float64(-0.10500325629011656),
  'x_pos': np.float64(1841.4),
  'energy': np.float32(0.0017542485),
  'momentum': np.float32(1.0804101),
  'theta': np.float32(89.911606),
  'time': np.float32(6.169733),
  'num_pixels': np.float64(29574.861555061914),
  'layer_energy': np.float32(0.006338837)},
 2: {'z_pos': np.float64(-1.5018469744955696),
  'x_pos': np.float64(1907.5),
  'energy': np.float32(0.0017355052),
  'momentum': np.float32(0.98359793),
  'theta': np.float32(90.94318),
  'time': np.float32(6.3937635),
  'num_pixels': np.float64(10666.185668974507),
  'layer_energy': np.float32(0.002291675)},
 3: {'z_pos': np.float64(-1.6755419435419436),
  'x_pos': np.flo

In [96]:
count = 0
for i in range(5000):
    if(0 in processed_data[i]):
        count += 1
print(count / 5000)

0.7618
